In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import numpy as np
import re

In [2]:
#                                         DATA
y, s = 34, 4
max_els = 100 # max elements in page
titles = np.zeros(y * s * max_els, dtype=object)
synopsis = np.zeros(y * s * max_els, dtype=object)
episodes = np.zeros(y * s * max_els, dtype=object)
aired = np.zeros(y * s * max_els, dtype=object)
seasons = np.zeros(y * s * max_els, dtype=object)
broadcasts = np.zeros(y * s * max_els, dtype=object)
studios = np.zeros(y * s * max_els, dtype=object)
sources = np.zeros(y * s * max_els, dtype=object)
genres = np.zeros(y * s * max_els, dtype=object)
demographics = np.zeros(y * s * max_els, dtype=object)
themes = np.zeros(y * s * max_els, dtype=object)
durations = np.zeros(y * s * max_els, dtype=object)
ratings = np.zeros(y * s * max_els, dtype=object)
scores = np.zeros(y * s * max_els, dtype=object)
scores_amount = np.zeros(y * s * max_els, dtype=object)
members = np.zeros(y * s * max_els, dtype=object)
favorites = np.zeros(y * s * max_els, dtype=object)


In [15]:
service = Service(executable_path='chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.get('https://myanimelist.net/anime/season')

count = 0
jump_to_season = -1
year = 2024
filter1 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/span[2]')))
filter2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/span[3]')))
filter1.click()
filter2.click()

for i in range(y * s):  # take last 34 years meaning from means from 2024 to 1990
    tv_section = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/ul/li[2]')))
    tv_section.click()
    jump_to_season += 1
    if jump_to_season == 4: 
        jump_to_season = 0
        year -= 1
    jump_to_year = str(year)
        
    fields_cont = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[1]/ul/li[9]/form')))
    season_field = WebDriverWait(fields_cont, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'select')))
    year_field = WebDriverWait(fields_cont, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input')))
    go = WebDriverWait(fields_cont, 15).until(EC.element_to_be_clickable((By.CLASS_NAME, 'inputButton')))
    
    select = Select(season_field)
    select.select_by_index(jump_to_season)
    year_field.send_keys(jump_to_year)
    go.click()

    imgs = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'image')))
    for j, img in enumerate(imgs):
        add = 0

        WebDriverWait(driver, 15).until(lambda driver: img.is_enabled())
        seriesPage_link = img.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        driver.get(seriesPage_link)
        
        title = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'title-english')))
        description = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[itemprop='description']")))
        info_cont = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'leftside')))        
        
        titles[count] = title.text
        synopsis[count] = re.sub('[Written by MAL Rewrite]', '', description.text)

        info_els = WebDriverWait(info_cont, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'spaceit_pad')))
        
        episodes[count] = info_els[4].text.split(':')[1].strip()
        aired[count] = info_els[6].text.split(':')[1].strip().split(',')[0:2][0]
        seasons[count] = info_els[7].text.split(' ')[1]
        broadcasts[count] = info_els[8].text.split(':')[1].strip()
        studios[count] = info_els[11].text.split(':')[1].strip()
        sources[count] = info_els[12].text.split(':')[1].strip()
        genres[count] = info_els[13].text.split(':')[1].strip()
        # span_14 = info_els[14].find_elment(By.CLASS_NAME, 'dark_text')
        # span_15 = info_els[15].find_elment(By.CLASS_NAME, 'dark_text')
        if len(info_els) == 23:
            themes[count] = info_els[14].text.split(':')[1].strip()
            demographics[count] = info_els[15].text.split(':')[1].strip()
            add = 2
        elif info_els[14].text.split(':')[0] == 'Theme': 
            themes[count] = info_els[14].text.split(':')[1].strip()
            add = 1
        elif info_els[14].text.split(':')[0] == 'Demographic': 
            demographics[count] = info_els[14].text.split(':')[1].strip()
            add = 1
        # else

        durations[count] = info_els[14+add].text.split(' ')[1]
        ratings[count] = info_els[15+add].text.split(':')[1].strip()
        scores[count] = info_els[16+add].text.split(' ')[1]
        try: scores_amount[count] = info_els[16+add].text.split(' ')[4]
        except Exception: pass
        members[count] = info_els[19+add].text.split(' ')[1]
        print(add)
        favorites[count] = info_els[20+add].text.split(' ')[1]
        
        
        count += 1
        driver.back()

1
0
2
0
1
1
2
2
0
2
2
1
1
1
0
0
0
0


IndexError: list index out of range